In [1]:
import json
import yaml
import time
import pandas as pd
import numpy as np
import datetime
import os
import sys
import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from openai import OpenAI
from dateutil import parser
import json
import re
from collections import Counter
from tqdm import tqdm
import pymssql
from threading import Thread
import functools
import difflib

# GPT API key
with open('openai_apikey.txt', 'r') as file:
    apikey = file.read()
os.environ["OPENAI_API_KEY"] = apikey

def calc_metrics(ground_truth, predictions):
    ground_truth_counter = Counter(ground_truth)
    predictions_counter = Counter(predictions)

    true_positives = sum((ground_truth_counter & predictions_counter).values())
    false_positives = sum((predictions_counter - ground_truth_counter).values())
    false_negatives = sum((ground_truth_counter - predictions_counter).values())

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

def evaluate(masked, generated):
    """ 
    Input: 
        - masked (str): Ground_truth text
        - generated(str): Text to be evaluated

    Output:
        - Precision, Recall and F1 (float)
    """
    ground_truth = re.findall(r'\[\*\*(.*?)\*\*\]', masked)
    predictions = re.findall(r'\[\*\*(.*?)\*\*\]', generated)
    #print(ground_truth)
    #print(predictions)
    
    return calc_metrics(ground_truth, predictions)

def highlight_deleted_parts(str1, str2):
    i = 0
    j = 0
    result = []
    deleted_part = ""

    while i < len(str1) and j < len(str2):
        if str1[i] == str2[j]:
            if deleted_part:
                result.append(f"[**{deleted_part}**]")
                deleted_part = ""
            result.append(str1[i])
            i += 1
            j += 1
        else:
            deleted_part += str1[i]
            i += 1

    # Remaining deleted characters in str1
    while i < len(str1):
        deleted_part += str1[i]
        i += 1

    if deleted_part:
        result.append(f"[**{deleted_part}**]")

    return ''.join(result)

def levenshtein_distance(s1, s2, show_progress=True):
    """
    Calcula la distancia de Levenshtein entre dos cadenas.

    La distancia de Levenshtein es el número mínimo de operaciones de edición 
    (inserción, eliminación o sustitución de un carácter) necesarias para 
    transformar una cadena en otra.

    Parámetros:
        s1 (str): Primera cadena
        s2 (str): Segunda cadena
        show_progress (bool): Si es True, muestra una barra de progreso. 
                              Por defecto es False.
    Retorna:
        int: La distancia de Levenshtein entre s1 y s2
    """
    # Usar tqdm solo si show_progress es True
    iterable = tqdm(s1) if show_progress else s1

    if len(s1) < len(s2):
        s1, s2 = s2, s1
    if len(s2) == 0:
        return len(s1)

    previous_row = list(range(len(s2) + 1))
    for i, c1 in enumerate(iterable):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

CARMEN_base_path='CARMEN-I_v1.01b 2/CARMEN-I_v1.01b/txt/'

txtlist=os.listdir(CARMEN_base_path+'masked/')

# This is for connecting to the MSSQL server via pymssql. Adjust it according to your configuration.
conn = pymssql.connect(host=r"(local)", database='Ddrive5', charset='utf8')

# This is the table name that you will create in the MSSQL server to save the GPT responses.
# Adjust it according to your preferences.
newtablename='20240715_test_Prediction'

In [2]:
df=pd.DataFrame(columns=['txtname','Replaced','Masked','Replaced_enclosed'])

for i in range(len(txtlist)): 
    with open(CARMEN_base_path+'replaced/'+txtlist[i], 'r', encoding='utf-8') as file:
        Replaced = file.read()
        if Replaced[0]=='\n':
            Replaced=Replaced[1:]
    with open(CARMEN_base_path+'masked/'+txtlist[i], 'r', encoding='utf-8') as file:
        Masked = file.read()
        if Masked[0]=='\n':
            Masked=Masked[1:]
            
    pattern = r'\[\*\*.*?\*\*\]'
    Masked_deleted = re.sub(pattern, '', Masked)

    Replaced_enclosed = highlight_deleted_parts(Replaced, Masked_deleted)
    
    templist=[]
    templist.append(txtlist[i])
    templist.append(Replaced) 
    templist.append(Masked) 
    templist.append(Replaced_enclosed) 
    df.loc[len(df)]=templist

In [3]:
for newtablename in [newtablename]:
    sql_createtable="CREATE TABLE [" + newtablename +"""] 
    (
        txtname    NVARCHAR(max),
        txt   NVARCHAR(max),
    )

    """
    conn = pymssql.connect(host=r"(local)", database='Ddrive5', charset='utf8')
    with conn:
        with conn.cursor() as cur:
            cur.execute(sql_createtable)
            conn.commit()

In [4]:
def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco

            
@timeout(0.1)
def SQL_Prediction(Replaced, txtname, system_content, model_ver, numbers, temperature_setting):
    
    client = OpenAI()
    response = client.chat.completions.create(
        model=model_ver,
        n=numbers,
        temperature=temperature_setting,
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": [
                {"type": "text", "text": Replaced},
            ]}
        ],
    )
    
    Prediction=response.choices[0].message.content

    conn = pymssql.connect(host=r"(local)", database='Ddrive5', charset='utf8')
    sql = "INSERT INTO [" + newtablename+"] (txtname, txt) VALUES (%s, %s)"

    with conn:
        with conn.cursor() as cur:
            cur.execute(sql, (txtname, Prediction))
            conn.commit()

system_content2='''You are an anonimization tool in identifying attributes in texts that can identify or quasi-identify a user.
Return only the original text with the identification and labeling of the patient's personal information by adding it between [** and **].
For example, names such as John Doe should be given as [**John Doe**].
Dates such as 2024/07/01 should be given as [**2024/07/01**].
Other information such as the sex, words that can identify a person's sex, occupation, place of residence, contact information, identifiable numbers, medical history , etc should also be anonymized.
Do not comment anything else.
'''

In [5]:
for i in range(500):
    txtname=df['txtname'][i]
    Replaced=df['Replaced'][i]
    SQL_Prediction(Replaced, txtname, system_content2, 'gpt-4o', 1, 1.0)

In [13]:
conn = pymssql.connect(host=r"(local)", database='Ddrive5', charset='utf8')
sql_statement="select * from ["+ newtablename + "]"
df_SQL_Prediction = pd.read_sql(sql=sql_statement, con=conn)

C:\Users\Desktop\AppData\Local\Temp\ipykernel_27852\3136040221.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL_Prediction = pd.read_sql(sql=sql_statement, con=conn)


In [14]:
df_SQL_Prediction

,txtname,txt
0,CARMEN-I_IA_ANTECEDENTES_108.txt,"Paciente de [**37**] años, Alergia a Quinolona..."
1,CARMEN-I_IA_ANTECEDENTES_119.txt,[**Hombre**] de [**51 años**] originario de [*...
2,CARMEN-I_IA_ANTECEDENTES_10.txt,ANTECEDENTES:\n- No alergias conocidas\n- No h...
3,CARMEN-I_IA_ANTECEDENTES_106.txt,"Paciente de [**28**] años, sin alergias ni háb..."
4,CARMEN-I_IA_ANTECEDENTES_139.txt,Antecedentes personales: [**niega**]\nAlergias...
...,...,...
495,CARMEN-I_IA_EVOL_55.txt,Frotis + Covid-19. Estable HDM y respiratoriam...
496,CARMEN-I_IA_PROCESO_ACTUAL_34.txt,Consultó el [**21/07/2019**] al [**Hospital Cl...
497,CARMEN-I_IA_EVOL_94.txt,**EVOLUCIÓN EN SALA [**W053**] (PARTE 1)\n.\n1...
498,CARMEN-I_IA_EVOL_75.txt,* NOTA DE TRASLLAT DE LA [**J119**] A SALA CON...


In [15]:
while True:
    for i in range(500):
        if df['txtname'][i] not in list(df_SQL_Prediction['txtname']):
            txtname=df['txtname'][i]
            Replaced=df['Replaced'][i]
            SQL_Prediction(Replaced, txtname, system_content2, 'gpt-4o', 1, 1.0)
            print('inserted')
    time.sleep(60)
    print('slept')
    cnt=0
    for i in range(500):
        if df['txtname'][i] not in list(df_SQL_Prediction['txtname']):
            cnt+=1
    if cnt==0:
        break

slept


In [16]:
df2=pd.merge(df,df_SQL_Prediction,left_on='txtname',right_on='txtname',how='inner')
df2.columns=['txtname','Replaced','Masked','Replaced_enclosed','Prediction']

In [17]:
precision=[]
recall=[]
f1=[]
for i in range(len(df2)):
    cal_met = evaluate(df2['Replaced_enclosed'][i], df2['Prediction'][i])
    precision.append(cal_met[0])
    recall.append(cal_met[1])
    f1.append(cal_met[2])

df2['precision']=precision
df2['recall']=recall
df2['f1']=f1

In [18]:
df2

,txtname,Replaced,Masked,Replaced_enclosed,Prediction,precision,recall,f1
0,CARMEN-I_CC_1.txt,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,0.800000,0.750000,0.774194
1,CARMEN-I_CC_2.txt,Realizo llamada telefónica. Refiere buen desca...,Realizo llamada telefónica. Refiere buen desca...,Realizo llamada telefónica. Refiere buen desca...,Realizo llamada telefónica. Refiere buen desca...,0.833333,0.750000,0.789474
2,CARMEN-I_CC_3.txt,Visita unidad del dolor. Ver informe de evoluc...,Visita unidad del dolor. Ver informe de evoluc...,Visita unidad del dolor. Ver informe de evoluc...,Visita unidad del dolor. Ver informe de evoluc...,0.500000,0.666667,0.571429
3,CARMEN-I_CC_4.txt,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,0.833333,0.789474,0.810811
4,CARMEN-I_CC_5.txt,Primera llamada HDOM COVID-19 POSITIVO\nTrabaj...,Primera llamada HDOM COVID-19 POSITIVO\nTrabaj...,Primera llamada HDOM COVID-19 POSITIVO\nTrabaj...,Primera llamada HDOM COVID-19 POSITIVO\nTrabaj...,0.869565,0.800000,0.833333
...,...,...,...,...,...,...,...,...
495,CARMEN-I_IA_PROCESO_ACTUAL_57.txt,Presenta desde hace 3 dias (15.04.27) cuadro d...,Presenta desde hace 3 dias ([**FECHAS**]) cuad...,Presenta desde hace 3 dias ([**15.04.27**]) cu...,Presenta desde hace 3 dias ([**15.04.27**]) cu...,1.000000,1.000000,1.000000
496,CARMEN-I_IA_PROCESO_ACTUAL_58.txt,Paciente con antecedente de colelitiasis y rea...,Paciente con antecedente de colelitiasis y rea...,Paciente con antecedente de colelitiasis y rea...,Paciente con antecedente de colelitiasis y rea...,0.000000,0.000000,0.000000
497,CARMEN-I_IA_PROCESO_ACTUAL_59.txt,Paciente con los antecedentes previamente desc...,Paciente con los antecedentes previamente desc...,Paciente con los antecedentes previamente desc...,Paciente con los antecedentes previamente desc...,0.500000,0.333333,0.400000
498,CARMEN-I_IA_PROCESO_ACTUAL_6.txt,Explica molestia en region umbilical con poste...,Explica molestia en region umbilical con poste...,Explica molestia en region umbilical con poste...,Explica molestia en region umbilical con poste...,0.333333,0.500000,0.400000


In [23]:
df2.to_csv('20240715_CARMEN_results.csv',index=False,encoding='utf8')

In [48]:
a=5
print(df2['precision'][a])
print(df2['recall'][a])

0.5
0.36363636363636365


In [49]:
print(df2['Masked'][a])

[**SEXO_SUJETO_ASISTENCIA**] de [**EDAD_SUJETO_ASISTENCIA**], sin alergias a medicamentos conocidas. Exfumador desde hace 2 meses (previo de 1 paquete/día). Consumo de 2 UBE/día.
* VALORACIÓN GERIÁTRICA:
- FUNCIONAL: Barthel 100/100. Independiente para las actividades básicas de la vida diaria. Portador de BIPAP y O2 domiciliario.
-COGNITIVO: Funciones superiores conservadas. Pfeiffer sin errores.
- SOCIAL: Vive con su [**FAMILIARES_SUJETO_ASISTENCIA**]. Teleasistencia en tramite.
* ANTECEDENTES PERSONALES:
1. HIPERTENSIÓN ARTERIAL en tratamiento farmacológico.
2. FIBRILACIÓN AURICULAR PERMANENTE en tratamiento anticogulante con acenocumarol hasta [**FECHAS**] hasta hace dos semanas (posteriormente cambio a warfarina con último INR de 4 el [**FECHAS**]) y tratamiento cronótropo negativo con digoxina y diltiazem.
3. CARDIOPATÍA VALVULAR-HIPERTRÓFICA (IAo + EAo en límite severidad + IM ligera-moderada) con FEVI preservada que condiciona HIPERTENSIÓN PULMONAR SECUNDARIA GRAVE. CF NYHA II-

In [50]:
print(df2['Replaced_enclosed'][a])

[**Varón**] de [**79 años**], sin alergias a medicamentos conocidas. Exfumador desde hace 2 meses (previo de 1 paquete/día). Consumo de 2 UBE/día.
* VALORACIÓN GERIÁTRICA:
- FUNCIONAL: Barthel 100/100. Independiente para las actividades básicas de la vida diaria. Portador de BIPAP y O2 domiciliario.
-COGNITIVO: Funciones superiores conservadas. Pfeiffer sin errores.
- SOCIAL: Vive con su [**esposa**]. Teleasistencia en tramite.
* ANTECEDENTES PERSONALES:
1. HIPERTENSIÓN ARTERIAL en tratamiento farmacológico.
2. FIBRILACIÓN AURICULAR PERMANENTE en tratamiento anticogulante con acenocumarol hasta [**03/2027**] hasta hace dos semanas (posteriormente cambio a warfarina con último INR de 4 el [**1/05**]) y tratamiento cronótropo negativo con digoxina y diltiazem.
3. CARDIOPATÍA VALVULAR-HIPERTRÓFICA (IAo + EAo en límite severidad + IM ligera-moderada) con FEVI preservada que condiciona HIPERTENSIÓN PULMONAR SECUNDARIA GRAVE. CF NYHA II-III.
*ECOCARDIOGRAMA [**04/2027**]: Doble lesión aórtic

In [51]:
print(df2['Prediction'][a])

Varón de [**79**] años, sin alergias a medicamentos conocidas. Exfumador desde hace [**2 meses**] (previo de 1 paquete/día). Consumo de 2 UBE/día.
* VALORACIÓN GERIÁTRICA:
- FUNCIONAL: Barthel 100/100. Independiente para las actividades básicas de la vida diaria. Portador de BIPAP y O2 domiciliario.
- COGNITIVO: Funciones superiores conservadas. Pfeiffer sin errores.
- SOCIAL: Vive con [**su esposa**]. Teleasistencia en trámite.
* ANTECEDENTES PERSONALES:
1. HIPERTENSIÓN ARTERIAL en tratamiento farmacológico.
2. FIBRILACIÓN AURICULAR PERMANENTE en tratamiento anticoagulante con acenocumarol hasta [**03/2027**] hasta hace [**dos semanas**] (posteriormente cambio a warfarina con último INR de [**4 el 1/05**]) y tratamiento cronótropo negativo con digoxina y diltiazem.
3. CARDIOPATÍA VALVULAR-HIPERTRÓFICA (IAo + EAo en límite severidad + IM ligera-moderada) con FEVI preservada que condiciona HIPERTENSIÓN PULMONAR SECUNDARIA GRAVE. CF NYHA II-III.
*ECOCARDIOGRAMA [**04/2027**]: Doble lesió

In [57]:
levenshtein_distance(df2['Prediction'][1].replace('[**', '').replace('**]', ''),df2['Replaced'][1])

100%|████████████████████████████████████████████████████████████████████████████| 3411/3411 [00:02<00:00, 1270.87it/s]


1